In [1]:
#here ways of dealing with non-existing data and multiple values for a single data-point are developed

In [2]:
import h5py
import numpy as np
import pandas as pd 
import math

In [53]:
voxel_size = 3
max_x_part = 4
max_y_part = 4

In [4]:
def get_2D_data_from_h5(h5_path, part_name, Slice_name):
    with h5py.File(h5_path,'r') as h5:
        X_Axis = h5[part_name][Slice_name]['X_Axis']
        Y_Axis = h5[part_name][Slice_name]['Y_Axis']
        Area = h5[part_name][Slice_name]['Area']

        help_arr = np.column_stack((X_Axis, Y_Axis, Area))
        df = pd.DataFrame(help_arr, columns=['x','y','area'])
        return df

In [5]:
df = get_2D_data_from_h5("/home/jan/Documents/Voxel_Erstellung/example_hdf_3.hdf",'part_1','Slice_1')

In [7]:
#deleting a certain value of the dataframe to simulate a missing value (x=0, y=2)
df = df.drop(df.index[0])

In [19]:
#creating multiple values for one point by replacing the 0s with 1s 
df = df.replace(0,1)

In [63]:
#Berechnung der Anzahl an Voxeln (hier simpler Fall)
def get_number_voxel (max_x_part, max_y_part, voxel_size):
    n_voxels_x = math.ceil(int(max_x_part)/voxel_size)
    n_voxels_y = math.ceil(int(max_y_part)/voxel_size)
    return n_voxels_x, n_voxels_y #tuple is returned 

In [50]:
def fill_2D_voxel_area (voxel_size, num_voxels_x, num_voxels_y, df, filling_method):
    array_area = np.zeros([voxel_size,voxel_size]) #creating an empty array of dimensions voxel_size*voxel_size
    for i in range(voxel_size): #iterating over x
        for j in range(voxel_size): #iterating over y

            if df[(df['x'] == i) & (df['y'] == j)].shape[0] == 1: #here subset of the original dataframe is created an filtrered --> shape[0] returns the number of rows of this df subset  
                #finding the area value for a certain point in the part-data-dataframe and allocating it to a position in the array
                area_i = df.loc[(df['x'] == i) & (df['y'] == j)]
                array_area[i][j] = area_i['area']
            
            elif df[(df['x'] == i) & (df['y'] == j)].shape[0] > 1:
                #if there are more values than just one the maximum value is used for the voxel-datapoint; other methods of dealing with multiple values need to be considered 
                array_area[i][j] = df[(df['x'] == i) & (df['y'] == j)]['area'].max()

            
            elif df[(df['x'] == i) & (df['y'] == j)].shape[0] == 0 and filling_method == 'Zeros':
                array_area[i][j] = 0
                    
            #elif filling_method == 'Mean': #with this method all the missing datapoints are getting filled with the mean of the non-missing datapoints 
             #   array_area[i][j] =      

    return array_area

#subsetDataFrame = dfObj[dfObj['Product'].isin(['Mangos', 'Grapes']) ]
#df[df['first_name'].notnull() & (df['nationality'] == "USA")]
#movies[(movies.duration >= 200) & (movies.genre == 'Drama')]

In [45]:
new_df = df[(df['x'] == 1) & (df['y'] ==1)]
new_df.shape[0]

3

In [51]:
fill_2D_voxel_area(voxel_size, n_voxels[0], n_voxels[1],df,'Zeros')

array([[ 0.,  0.],
       [ 0., 34.]])